# ESP32 Model Export
แปลง ML model เป็น C++ header สำหรับ ESP32

## ทำไมต้อง retrain?
| | AutoGluon model | Small RF (ESP32) |
|--|--|--|
| Trees | 300 | 20 |
| Size | 140-380 MB | < 100 KB |
| Runs on ESP32 | ❌ | ✅ |

## แผน
1. โหลดข้อมูล → extract features เหมือน notebook หลัก
2. Train **RandomForestClassifier ขนาดเล็ก** (n_estimators=20, max_depth=10)
3. Export แต่ละ mode เป็น `.h` header file ด้วย **micromlgen**
4. ดู Arduino code ที่ต้องแก้

> **หมายเหตุ**: micromlgen รองรับ `RandomForestClassifier` เท่านั้น (ไม่รองรับ ExtraTrees)

In [9]:
import os, glob, pickle, warnings
import numpy as np
import pandas as pd
from scipy import stats
from scipy.fft import fft, fftfreq
from sklearn.ensemble import RandomForestClassifier   # micromlgen รองรับ RF เท่านั้น
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, classification_report
from micromlgen import port
warnings.filterwarnings('ignore')

# ── Constants (same as mode notebooks) ───────────────────────────────────────
SAMPLE_RATE  = 50
WINDOW_SIZE  = 100
STEP_SIZE    = 50
ADXL_SCALE   = 32.0 / 8192.0
ITG_SCALE    = 4000.0 / 65536.0
RANDOM_STATE = 42

FEATURE_NAMES_23 = [
    'mean_ax','mean_ay','mean_az','std_ax','std_ay','std_az',
    'min_mag','max_mag','range_mag','rms_ax','rms_ay','rms_az',
    'skewness','kurtosis','zero_cross','SMA',
    'dom_freq','spectral_energy','corr_xy','corr_yz','corr_xz',
    'max_jerk','acc_variance',
]
FEATURE_NAMES_26 = FEATURE_NAMES_23 + ['step_freq','gait_regularity','vertical_symmetry']

os.makedirs('models/esp32', exist_ok=True)
print('Config ready. Output folder: models/esp32/')
print('Classifier: RandomForestClassifier (n_estimators=20, max_depth=10)')

Config ready. Output folder: models/esp32/
Classifier: RandomForestClassifier (n_estimators=20, max_depth=10)


## 1. Feature Extraction Functions

In [10]:
def extract_23(window, fs=SAMPLE_RATE):
    ax, ay, az = window[:,0], window[:,1], window[:,2]
    mag   = np.sqrt(ax**2+ay**2+az**2)
    mag_c = mag - np.mean(mag)
    freqs = fftfreq(len(mag), 1/fs)
    fmag  = np.abs(fft(mag))[:len(mag)//2]
    pf    = freqs[:len(mag)//2]
    jerk  = np.diff(mag)*fs
    f  = [float(np.mean(ax)),float(np.mean(ay)),float(np.mean(az))]
    f += [float(np.std(ax)), float(np.std(ay)), float(np.std(az))]
    f += [float(np.min(mag)),float(np.max(mag)),float(np.ptp(mag))]
    f += [float(np.sqrt(np.mean(ax**2))),float(np.sqrt(np.mean(ay**2))),float(np.sqrt(np.mean(az**2)))]
    f += [float(stats.skew(mag)),float(stats.kurtosis(mag))]
    f.append(float(np.sum(np.diff(np.sign(mag_c))!=0)))
    f.append(float(np.sum(np.abs(ax)+np.abs(ay)+np.abs(az))/len(ax)))
    f.append(float(pf[np.argmax(fmag)]) if len(fmag)>0 else 0.0)
    f.append(float(np.sum(fmag**2)))
    f.append(float(np.corrcoef(ax,ay)[0,1]) if np.std(ax)>0 and np.std(ay)>0 else 0.0)
    f.append(float(np.corrcoef(ay,az)[0,1]) if np.std(ay)>0 and np.std(az)>0 else 0.0)
    f.append(float(np.corrcoef(ax,az)[0,1]) if np.std(ax)>0 and np.std(az)>0 else 0.0)
    f.append(float(np.max(np.abs(jerk))) if len(jerk)>0 else 0.0)
    f.append(float(np.var(mag)))
    return np.array(f, dtype=np.float32)

def extract_26(window, fs=40):
    base = extract_23(window, fs)
    ax, ay, az = window[:,0], window[:,1], window[:,2]
    mag=np.sqrt(ax**2+ay**2+az**2); mag_c=mag-np.mean(mag)
    freqs=fftfreq(len(mag),1/fs); fmag=np.abs(fft(mag))[:len(mag)//2]
    pf=freqs[:len(mag)//2]; gm=(pf>=0.5)&(pf<=3.0)
    sf=float(pf[gm][np.argmax(fmag[gm])]) if (gm.any() and len(fmag)==len(pf)) else 0.0
    ac=np.correlate(mag_c,mag_c,mode='full')[len(mag_c)-1:]; ac=ac/(ac[0]+1e-9)
    lmin,lmax=int(0.4*fs),int(1.2*fs)
    gr=float(ac[lmin:lmax].max()) if lmax<len(ac) else 0.0
    vs=float(np.std(az)/(np.std(ax)+np.std(ay)+1e-9))
    return np.concatenate([base,[sf,gr,vs]]).astype(np.float32)

def build_features(raw_df, extractor, feat_names, win=WINDOW_SIZE, step=STEP_SIZE):
    rows = []
    group_col = 'file' if 'file' in raw_df.columns else None
    groups = raw_df.groupby(group_col) if group_col else [(None, raw_df)]
    for _, grp in groups:
        data   = grp[['ax','ay','az','gx','gy','gz']].values.astype(np.float32)
        labels = grp['label'].values
        for s in range(0, len(data)-win, step):
            w = data[s:s+win]
            rows.append(np.append(extractor(w), int(labels[s:s+win].mean()>0.5)))
    return pd.DataFrame(rows, columns=feat_names+['label'])

def build_features_fallalld(df_device, extractor, feat_names, win=80, step=40):
    rows = []
    for _, row in df_device.iterrows():
        acc=row['Acc'].astype(np.float32); lbl=int(row['label'])
        for s in range(0, len(acc)-win, step):
            rows.append(np.append(extractor(acc[s:s+win]), lbl))
    return pd.DataFrame(rows, columns=feat_names+['label'])

print('Feature extractors ready.')

Feature extractors ready.


## 2. Train Small Models + Export C++ Headers

In [11]:
def train_small_and_export(feat_df, feat_names, mode, classname):
    """Train RandomForest (n_estimators=20, max_depth=10) → export C++ header."""
    X = feat_df[feat_names].values
    y = feat_df['label'].values
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

    # Small RandomForest for ESP32 (micromlgen supports RF, not ExtraTrees)
    clf = RandomForestClassifier(
        n_estimators  = 20,
        max_depth     = 10,
        max_leaf_nodes= 80,
        class_weight  = 'balanced',
        random_state  = RANDOM_STATE,
        n_jobs        = -1,
    )
    clf.fit(X_tr, y_tr)

    y_pred = clf.predict(X_te)
    y_prob = clf.predict_proba(X_te)[:,1]
    f1  = f1_score(y_te, y_pred)
    auc = roc_auc_score(y_te, y_prob)

    # Node count & size estimate
    total_nodes = sum(t.tree_.node_count for t in clf.estimators_)
    n_trees     = len(clf.estimators_)

    print(f'=== {mode} Small Model ===')
    print(f'  F1={f1:.3f}  AUC={auc:.3f}')
    print(f'  Total nodes: {total_nodes:,}  avg/tree: {total_nodes//n_trees}')

    # Export C++ with micromlgen (RandomForestClassifier supported)
    c_code = port(clf, classname=classname, optimize=False)
    header_path = f'models/esp32/{mode}_model.h'
    with open(header_path, 'w') as f:
        f.write(c_code)
    size_kb = os.path.getsize(header_path) / 1024
    print(f'  Header size: {size_kb:.1f} KB → {header_path}')
    print()
    return clf, f1, auc, size_kb

results_small = {}

### 2a. CHEST (SisFall)

In [12]:
print('Loading SisFall...')
sisfall_records = []
for path in glob.glob('data/SisFall/**/*.txt', recursive=True):
    fname = os.path.basename(path)
    if fname.lower().startswith('readme'):
        continue
    label = 1 if fname.upper().startswith('F') else 0
    try:
        raw = pd.read_csv(path, header=None, sep=',')
        if raw.shape[1] < 6: continue
        raw.iloc[:,-1] = raw.iloc[:,-1].astype(str).str.replace(';','',regex=False).str.strip()
        raw = raw.apply(pd.to_numeric, errors='coerce').dropna()
        raw = raw.iloc[::4].reset_index(drop=True)
        df = pd.DataFrame({
            'ax': raw.iloc[:,0]*ADXL_SCALE, 'ay': raw.iloc[:,1]*ADXL_SCALE,
            'az': raw.iloc[:,2]*ADXL_SCALE, 'gx': raw.iloc[:,3]*ITG_SCALE,
            'gy': raw.iloc[:,4]*ITG_SCALE,  'gz': raw.iloc[:,5]*ITG_SCALE,
            'label': label, 'file': fname,
        })
        sisfall_records.append(df)
    except Exception: pass

sisfall_raw = pd.concat(sisfall_records, ignore_index=True)
print(f'SisFall: {len(sisfall_raw):,} samples | falls: {(sisfall_raw.label==1).sum():,}')
print('Extracting CHEST features...')
chest_df = build_features(sisfall_raw, extract_23, FEATURE_NAMES_23)
print(f'Windows: {len(chest_df):,}')

clf_chest, f1c, aucc, szc = train_small_and_export(chest_df, FEATURE_NAMES_23, 'CHEST', 'FallDetectorChest')
results_small['CHEST'] = {'f1': f1c, 'auc': aucc, 'size_kb': szc}

Loading SisFall...
SisFall: 3,964,862 samples | falls: 1,348,463
Extracting CHEST features...
Windows: 70,346
=== CHEST Small Model ===
  F1=0.764  AUC=0.910
  Total nodes: 3,180  avg/tree: 159
  Header size: 475.4 KB → models/esp32/CHEST_model.h



### 2b. SHIRT + PANTS (FallAllD)

In [13]:
print('Loading FallAllD...')
with open('data/FallAllD/FallAllD_40SamplesPerSec_ActivityIdsFiltered.pkl','rb') as f:
    fallalld = pickle.load(f)
df_waist = fallalld[fallalld['Device']=='Waist'].copy()
df_waist['label'] = (df_waist['ActivityID'] >= 101).astype(int)
print(f'FallAllD Waist: {len(df_waist):,} rows')

print('Extracting SHIRT features (23)...')
shirt_df = build_features_fallalld(df_waist, extract_23, FEATURE_NAMES_23)
clf_shirt, f1s, aucs, szs = train_small_and_export(shirt_df, FEATURE_NAMES_23, 'SHIRT', 'FallDetectorShirt')
results_small['SHIRT'] = {'f1': f1s, 'auc': aucs, 'size_kb': szs}

print('Extracting PANTS features (26)...')
pants_df = build_features_fallalld(df_waist, extract_26, FEATURE_NAMES_26, win=80, step=40)
clf_pants, f1p, aucp, szp = train_small_and_export(pants_df, FEATURE_NAMES_26, 'PANTS', 'FallDetectorPants')
results_small['PANTS'] = {'f1': f1p, 'auc': aucp, 'size_kb': szp}

Loading FallAllD...
FallAllD Waist: 1,706 rows
Extracting SHIRT features (23)...
=== SHIRT Small Model ===
  F1=0.687  AUC=0.911
  Total nodes: 3,180  avg/tree: 159
  Header size: 475.6 KB → models/esp32/SHIRT_model.h

Extracting PANTS features (26)...
=== PANTS Small Model ===
  F1=0.685  AUC=0.914
  Total nodes: 3,180  avg/tree: 159
  Header size: 476.3 KB → models/esp32/PANTS_model.h



## 3. Summary — Small vs Full Model

In [14]:
full = {}
for p in ['CHEST','SHIRT','PANTS']:
    row = pd.read_csv(f'models/{p}_results.csv').iloc[0]
    full[p] = {'f1': row['test_f1'], 'auc': row['auc']}

print(f'{'Mode':<8} | {'Full F1':>8} | {'Small F1':>9} | {'Full AUC':>9} | {'Small AUC':>10} | {'Size':>8}')
print('-'*65)
for p in ['CHEST','SHIRT','PANTS']:
    r = results_small[p]
    print(f'{p:<8} | {full[p]["f1"]:>8.3f} | {r["f1"]:>9.3f} | {full[p]["auc"]:>9.3f} | {r["auc"]:>10.3f} | {r["size_kb"]:>6.0f} KB')

print()
print('Headers saved to: models/esp32/')
for f in os.listdir('models/esp32'):
    sz = os.path.getsize(f'models/esp32/{f}')/1024
    print(f'  {f:<30} {sz:>6.1f} KB')

Mode     |  Full F1 |  Small F1 |  Full AUC |  Small AUC |     Size
-----------------------------------------------------------------
CHEST    |    0.869 |     0.764 |     0.968 |      0.910 |    475 KB
SHIRT    |    0.849 |     0.687 |     0.976 |      0.911 |    476 KB
PANTS    |    0.836 |     0.685 |     0.971 |      0.914 |    476 KB

Headers saved to: models/esp32/
  PANTS_model.h                   476.3 KB
  CHEST_model.h                   475.4 KB
  SHIRT_model.h                   475.6 KB


## 4. ดู C++ Header ที่ได้
ตัวอย่าง 10 บรรทัดแรกของแต่ละ header:

In [15]:
for mode in ['CHEST','SHIRT','PANTS']:
    path = f'models/esp32/{mode}_model.h'
    with open(path) as f:
        lines = f.readlines()[:12]
    print(f'=== {mode}_model.h ({len(open(path).readlines())} lines) ===')
    print(''.join(lines))
    print()

=== CHEST_model.h (9571 lines) ===
#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class FallDetectorChest {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        uint8_t votes[2] = { 0 };


=== SHIRT_model.h (9571 lines) ===
#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class FallDetectorShirt {
                public:
                    /**
                    * Predict class for features vector
                    */
                    int predict(float *x) {
                        uint8_t votes[2] = { 0 };


=== PANTS_model.h (9571 lines) ===
#pragma once
#include <cstdarg>
namespace Eloquent {
    namespace ML {
        namespace Port {
            class FallDetectorPants {
                public:
                    

## 5. วิธีใช้ใน Arduino ESP32

### ขั้นตอน
1. Copy `models/esp32/CHEST_model.h`, `SHIRT_model.h`, `PANTS_model.h` → Arduino sketch folder
2. Install library: **ArduinoFFT** (Arduino IDE → Library Manager → search "ArduinoFFT")
3. แก้ไข `Full_System_README.ino` ตามที่แสดงใน cell ถัดไป

### Architecture บน ESP32
```
loop @ 50Hz (20ms)
    │
    ├─ อ่าน MPU6050 → เก็บใน circular buffer (100 samples)
    │
    ├─ ทุก 50 samples (1 วิ) → extract 23 features
    │
    └─ ส่งเข้า model ตาม currentMode
           CHEST → FallDetectorChest.predict(features)
           SHIRT → FallDetectorShirt.predict(features)
           PANTS → FallDetectorPants.predict(features)
```

In [16]:
arduino_code = '''
// ─── ส่วนที่เพิ่มใน Full_System_README.ino ───────────────────────────────────

// 1) Include headers (เพิ่มหลัง #include "config.h")
#include "CHEST_model.h"
#include "SHIRT_model.h"
#include "PANTS_model.h"
#include <arduinoFFT.h>

// 2) Circular buffer สำหรับ 100 samples @ 50Hz
#define ML_WIN    100
#define ML_FS     50
#define ML_STEP   50          // run inference ทุก 50 samples

float ax_buf[ML_WIN], ay_buf[ML_WIN], az_buf[ML_WIN];
int   buf_idx   = 0;
int   samples_since_last_infer = 0;

// 3) Feature extraction (ใส่ก่อน loop())
float ml_features[26];   // ใช้ 23 หรือ 26 ขึ้นกับ mode

void extract_features(float* ax, float* ay, float* az, int n, float* out, int fs=50) {
    float mag[ML_WIN];
    float sum_ax=0, sum_ay=0, sum_az=0;
    for(int i=0;i<n;i++){
        mag[i]=sqrt(ax[i]*ax[i]+ay[i]*ay[i]+az[i]*az[i]);
        sum_ax+=ax[i]; sum_ay+=ay[i]; sum_az+=az[i];
    }
    float mean_ax=sum_ax/n, mean_ay=sum_ay/n, mean_az=sum_az/n;
    float mag_mean=0; for(int i=0;i<n;i++) mag_mean+=mag[i]; mag_mean/=n;

    // std
    float var_ax=0,var_ay=0,var_az=0,var_mag=0;
    for(int i=0;i<n;i++){
        var_ax+=(ax[i]-mean_ax)*(ax[i]-mean_ax);
        var_ay+=(ay[i]-mean_ay)*(ay[i]-mean_ay);
        var_az+=(az[i]-mean_az)*(az[i]-mean_az);
        var_mag+=(mag[i]-mag_mean)*(mag[i]-mag_mean);
    }
    float std_ax=sqrt(var_ax/n), std_ay=sqrt(var_ay/n), std_az=sqrt(var_az/n);

    // min/max/range of mag
    float min_mag=mag[0],max_mag=mag[0];
    for(int i=1;i<n;i++){if(mag[i]<min_mag)min_mag=mag[i]; if(mag[i]>max_mag)max_mag=mag[i];}

    // RMS
    float rms_ax=0,rms_ay=0,rms_az=0;
    for(int i=0;i<n;i++){rms_ax+=ax[i]*ax[i]; rms_ay+=ay[i]*ay[i]; rms_az+=az[i]*az[i];}
    rms_ax=sqrt(rms_ax/n); rms_ay=sqrt(rms_ay/n); rms_az=sqrt(rms_az/n);

    // skewness, kurtosis (simplified)
    float skew=0,kurt=0;
    for(int i=0;i<n;i++){
        float d=(mag[i]-mag_mean);
        skew+=d*d*d; kurt+=d*d*d*d;
    }
    float std_mag=sqrt(var_mag/n)+1e-9;
    skew/=(n*std_mag*std_mag*std_mag);
    kurt/=(n*std_mag*std_mag*std_mag*std_mag);

    // zero-crossing
    int zc=0;
    for(int i=1;i<n;i++){
        if((mag[i]-mag_mean)*(mag[i-1]-mag_mean)<0) zc++;
    }

    // SMA
    float sma=0;
    for(int i=0;i<n;i++) sma+=fabs(ax[i])+fabs(ay[i])+fabs(az[i]);
    sma/=n;

    // FFT (ArduinoFFT) for dominant freq + spectral energy
    double vReal[ML_WIN], vImag[ML_WIN];
    for(int i=0;i<n;i++){vReal[i]=mag[i]; vImag[i]=0;}
    ArduinoFFT<double> FFT(vReal,vImag,n,fs);
    FFT.windowing(FFT_WIN_TYP_HAMMING, FFT_FORWARD);
    FFT.compute(FFT_FORWARD);
    FFT.complexToMagnitude();
    float dom_freq = FFT.majorPeak();
    float spec_e=0; for(int i=0;i<n/2;i++) spec_e+=vReal[i]*vReal[i];

    // Max jerk
    float max_jerk=0;
    for(int i=1;i<n;i++){
        float j=fabs(mag[i]-mag[i-1])*fs;
        if(j>max_jerk) max_jerk=j;
    }

    // Fill features array (indices match Python extract_23)
    int k=0;
    out[k++]=mean_ax; out[k++]=mean_ay; out[k++]=mean_az;
    out[k++]=std_ax;  out[k++]=std_ay;  out[k++]=std_az;
    out[k++]=min_mag; out[k++]=max_mag; out[k++]=max_mag-min_mag;
    out[k++]=rms_ax;  out[k++]=rms_ay;  out[k++]=rms_az;
    out[k++]=skew;    out[k++]=kurt-3;  // excess kurtosis
    out[k++]=zc;      out[k++]=sma;
    out[k++]=dom_freq; out[k++]=spec_e;
    // correlations (simplified Pearson)
    float cxy=0,cyz=0,cxz=0;
    for(int i=0;i<n;i++){
        cxy+=(ax[i]-mean_ax)*(ay[i]-mean_ay);
        cyz+=(ay[i]-mean_ay)*(az[i]-mean_az);
        cxz+=(ax[i]-mean_ax)*(az[i]-mean_az);
    }
    float dxy=(std_ax*std_ay*n)+1e-9, dyz=(std_ay*std_az*n)+1e-9, dxz=(std_ax*std_az*n)+1e-9;
    out[k++]=cxy/dxy; out[k++]=cyz/dyz; out[k++]=cxz/dxz;
    out[k++]=max_jerk;
    out[k++]=var_mag/n;   // acc_variance
}

// 4) ML inference call (ใส่ในส่วนที่รัน 50Hz)
//    เรียกแทน หรือเพิ่มเติมจาก threshold logic
int ml_predict() {
    extract_features(ax_buf, ay_buf, az_buf, ML_WIN, ml_features, ML_FS);
    if      (deviceMode == CHEST) return FallDetectorChest::predict(ml_features);
    else if (deviceMode == SHIRT) return FallDetectorShirt::predict(ml_features);
    else                          return FallDetectorPants::predict(ml_features);
}

// 5) ใน loop() — เปลี่ยน sampling rate เป็น 50Hz (เดิมเป็น 5Hz)
//    เพิ่มในส่วน loop() หลัง mpu.update():
void loop_ml_section() {   // เรียกแทน delay(200) เดิม
    mpu.update();
    ax_buf[buf_idx] = mpu.getAccX();
    ay_buf[buf_idx] = mpu.getAccY();
    az_buf[buf_idx] = mpu.getAccZ();
    buf_idx = (buf_idx + 1) % ML_WIN;
    samples_since_last_infer++;

    if (samples_since_last_infer >= ML_STEP) {
        samples_since_last_infer = 0;
        int pred = ml_predict();
        if (pred == 1) {
            // ML says FALL → ทำ emergency (แทน threshold)
            triggerEmergency("ML FALL");
        }
    }
    delay(1000/ML_FS);   // 20ms = 50Hz
}
'''
print(arduino_code)


// ─── ส่วนที่เพิ่มใน Full_System_README.ino ───────────────────────────────────

// 1) Include headers (เพิ่มหลัง #include "config.h")
#include "CHEST_model.h"
#include "SHIRT_model.h"
#include "PANTS_model.h"
#include <arduinoFFT.h>

// 2) Circular buffer สำหรับ 100 samples @ 50Hz
#define ML_WIN    100
#define ML_FS     50
#define ML_STEP   50          // run inference ทุก 50 samples

float ax_buf[ML_WIN], ay_buf[ML_WIN], az_buf[ML_WIN];
int   buf_idx   = 0;
int   samples_since_last_infer = 0;

// 3) Feature extraction (ใส่ก่อน loop())
float ml_features[26];   // ใช้ 23 หรือ 26 ขึ้นกับ mode

void extract_features(float* ax, float* ay, float* az, int n, float* out, int fs=50) {
    float mag[ML_WIN];
    float sum_ax=0, sum_ay=0, sum_az=0;
    for(int i=0;i<n;i++){
        mag[i]=sqrt(ax[i]*ax[i]+ay[i]*ay[i]+az[i]*az[i]);
        sum_ax+=ax[i]; sum_ay+=ay[i]; sum_az+=az[i];
    }
    float mean_ax=sum_ax/n, mean_ay=sum_ay/n, mean_az=sum_az/n;
    float mag_mean=0; for(int i=0;i<n;i++

## 6. สรุปขั้นตอนการใช้งาน

```
1. รัน cell ข้างบนทั้งหมด
   → ได้ไฟล์ models/esp32/CHEST_model.h
   → ได้ไฟล์ models/esp32/SHIRT_model.h
   → ได้ไฟล์ models/esp32/PANTS_model.h

2. Copy ไฟล์ .h ทั้ง 3 → Full_System_README/ (sketch folder)

3. Arduino IDE → Library Manager → install "ArduinoFFT"

4. เพิ่ม code จาก cell 5 ใน Full_System_README.ino

5. อัพโหลด → ทดสอบ
```

> **หมายเหตุ**: Small model (15 trees) มี accuracy ต่ำกว่า Full model
> แนะนำให้ใช้ **ร่วมกัน** กับ threshold เดิม:
> - threshold ตรวจแบบ state machine (เร็ว, ไม่ต้องการ window)  
> - ML ยืนยันผล (reduce false positive)